# Generating pitch tracks for the Guše visualizer

In [1]:
import json
import os
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
folderName = './'
mbid = '67d7a5d6-e3aa-431a-a1e1-68df78c2f1a8'
fileName = mbid + '.f0_fil.csv'

dur = 52

shahed = 219.382

filePath = os.path.join(folderName, fileName)
print(filePath)

./67d7a5d6-e3aa-431a-a1e1-68df78c2f1a8.f0_fil.csv


### From Hz to cents

In [3]:
def h2c(t, h):
    return np.rint(1200 * np.log2(h/t))

In [4]:
# Load csv
pitchTrackHz = np.genfromtxt(filePath, delimiter=",")
pitchTrackHz

array([[  0.74 , 195.564],
       [  0.75 , 196.081],
       [  0.76 , 195.892],
       ...,
       [ 51.69 , 147.341],
       [ 51.7  , 147.301],
       [ 51.71 , 147.515]])

In [5]:
maxCent = h2c(shahed, np.max(pitchTrackHz[:,1]))
print("Max:", maxCent)

minCent = h2c(shahed, np.min(pitchTrackHz[:,1]))
print("Min:", minCent)

Max: 791.0
Min: -1400.0


In [6]:
# minCent = -1399
minCut = False

In [7]:
# Prepare dic
pitchTrackDic = {}

for frame in pitchTrackHz:
    if minCut and frame[1] >= minCent:
        pitchTrackDic["{:.2f}".format(frame[0])] = frame[1]
    else:
        pitchTrackDic["{:.2f}".format(frame[0])] = frame[1]

In [8]:
# Find duration

if pitchTrackHz[-1,0] > dur:
    dur = pitchTrackHz[-1,0]

dur

52

In [9]:
# Cent pitch track

pitchTrackCents = {}

for i in range(dur * 100):
    k = "{:.2f}".format(i/100)
    if k not in pitchTrackDic.keys():
        pitchTrackCents[k] = "s"
    else:
        pitchTrackCents[k] = h2c(shahed, pitchTrackDic[k])

pitchTrackCents

{'0.00': 's',
 '0.01': 's',
 '0.02': 's',
 '0.03': 's',
 '0.04': 's',
 '0.05': 's',
 '0.06': 's',
 '0.07': 's',
 '0.08': 's',
 '0.09': 's',
 '0.10': 's',
 '0.11': 's',
 '0.12': 's',
 '0.13': 's',
 '0.14': 's',
 '0.15': 's',
 '0.16': 's',
 '0.17': 's',
 '0.18': 's',
 '0.19': 's',
 '0.20': 's',
 '0.21': 's',
 '0.22': 's',
 '0.23': 's',
 '0.24': 's',
 '0.25': 's',
 '0.26': 's',
 '0.27': 's',
 '0.28': 's',
 '0.29': 's',
 '0.30': 's',
 '0.31': 's',
 '0.32': 's',
 '0.33': 's',
 '0.34': 's',
 '0.35': 's',
 '0.36': 's',
 '0.37': 's',
 '0.38': 's',
 '0.39': 's',
 '0.40': 's',
 '0.41': 's',
 '0.42': 's',
 '0.43': 's',
 '0.44': 's',
 '0.45': 's',
 '0.46': 's',
 '0.47': 's',
 '0.48': 's',
 '0.49': 's',
 '0.50': 's',
 '0.51': 's',
 '0.52': 's',
 '0.53': 's',
 '0.54': 's',
 '0.55': 's',
 '0.56': 's',
 '0.57': 's',
 '0.58': 's',
 '0.59': 's',
 '0.60': 's',
 '0.61': 's',
 '0.62': 's',
 '0.63': 's',
 '0.64': 's',
 '0.65': 's',
 '0.66': 's',
 '0.67': 's',
 '0.68': 's',
 '0.69': 's',
 '0.70': 's',
 '0.71

In [10]:
with open(mbid + "_pitchTrack.json", 'w') as f:
    json.dump(pitchTrackCents, f)

### Pitch histogram

In [11]:
import intonation
import numpy as np
import json
import os

In [12]:
def pitch2cents(pitch, ref):
    '''
    Converts a pitch track in Hz to cents given a reference pitch
    
    Args:
        pitch (np.array): a numpy array with two columns, first for time,
                          second for pitch
        ref (float): the reference pitch
    
    Returns:
        cents (np.array) a numpy array with one column of pitch in cents
    '''

    return 1200 * np.log2(pitch[:,1] / float(ref))

In [13]:
pitchTrackFolder = './'
mbid = '67d7a5d6-e3aa-431a-a1e1-68df78c2f1a8'
pitchTrackExtension = '.f0_fil.csv'

shahed = 219.382

In [14]:
pitchTrack = np.genfromtxt(os.path.join(pitchTrackFolder, mbid + pitchTrackExtension), delimiter=',')
centsTrack = pitch2cents(pitchTrack, shahed)
# filteredTrack = centsTrack[np.logical_and(centsTrack >= lowCut, centsTrack <= highCut)]
# pitch_obj = intonation.Pitch(np.arange(len(filteredTrack)), filteredTrack)
pitch_obj = intonation.Pitch(np.arange(len(centsTrack)), centsTrack)
rec_obj = intonation.Recording(pitch_obj)
rec_obj.compute_hist()

print(rec_obj.histogram.x)
print(rec_obj.histogram.y)

[-1399.1912355  -1398.19117235 -1397.19110919 ...   788.94695348
   789.94701664   790.94707979]
[0.00051758 0.00050904 0.00049237 ... 0.00033827 0.00034508 0.00034853]


In [15]:
histDic = {}

for i in range(rec_obj.histogram.x.size):
    # pos = '{:.2f}'.format(rec_obj.histogram.x[i])
    x = round(rec_obj.histogram.x[i], 2)
    y = round(rec_obj.histogram.y[i] * 10000, 2)
    histDic[x] = y

histMin = np.min(list(histDic.values()))
histMax = np.max(list(histDic.values()))

print(histMin, histMax)

histList = []
for k, v in histDic.items():
    histList.append([k,v])

hist2json = {'min': histMin, 'max': histMax, 'hist': histList}
hist2json

0.0 123.84


{'min': 0.0,
 'max': 123.84,
 'hist': [[-1399.19, 5.18],
  [-1398.19, 5.09],
  [-1397.19, 4.92],
  [-1396.19, 4.68],
  [-1395.19, 4.38],
  [-1394.19, 4.03],
  [-1393.19, 3.65],
  [-1392.19, 3.24],
  [-1391.19, 2.83],
  [-1390.19, 2.43],
  [-1389.19, 2.06],
  [-1388.19, 1.71],
  [-1387.19, 1.39],
  [-1386.19, 1.11],
  [-1385.19, 0.88],
  [-1384.19, 0.68],
  [-1383.19, 0.51],
  [-1382.19, 0.38],
  [-1381.19, 0.28],
  [-1380.19, 0.2],
  [-1379.19, 0.14],
  [-1378.19, 0.1],
  [-1377.19, 0.07],
  [-1376.19, 0.04],
  [-1375.19, 0.03],
  [-1374.19, 0.02],
  [-1373.19, 0.01],
  [-1372.19, 0.01],
  [-1371.19, 0.0],
  [-1370.19, 0.0],
  [-1369.19, 0.0],
  [-1368.19, 0.0],
  [-1367.19, 0.0],
  [-1366.19, 0.0],
  [-1365.19, 0.0],
  [-1364.19, 0.0],
  [-1363.19, 0.0],
  [-1362.19, 0.0],
  [-1361.19, 0.0],
  [-1360.19, 0.0],
  [-1359.19, 0.0],
  [-1358.19, 0.0],
  [-1357.19, 0.0],
  [-1356.19, 0.0],
  [-1355.19, 0.0],
  [-1354.19, 0.0],
  [-1353.19, 0.0],
  [-1352.19, 0.0],
  [-1351.19, 0.0],
  [-13

In [16]:
pitchHistogramFolder = './'
pitchHistogramExtension = '_pitchHistogram.json'

json.dumps(hist2json)

with open(os.path.join(pitchHistogramFolder, mbid + pitchHistogramExtension), 'w') as f:
    json.dump(hist2json, f)